In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


IMPORTING LIBRARIES

In [2]:
import numpy as np 
import pandas as pd 
import tensorflow as tf

LOADING TRAIN SET

In [8]:
train_data = pd.read_csv('/content/drive/MyDrive/train.csv',nrows=3e6)
raw_ids_all = train_data["Id"]#only ids
raw_ids = raw_ids_all.unique()#unique ids
train_data.head(25)

,Id,minutes_past,radardist_km,Ref,Ref_5x5_10th,Ref_5x5_50th,Ref_5x5_90th,RefComposite,RefComposite_5x5_10th,RefComposite_5x5_50th,RefComposite_5x5_90th,RhoHV,RhoHV_5x5_10th,RhoHV_5x5_50th,RhoHV_5x5_90th,Zdr,Zdr_5x5_10th,Zdr_5x5_50th,Zdr_5x5_90th,Kdp,Kdp_5x5_10th,Kdp_5x5_50th,Kdp_5x5_90th,Expected
0,1,3,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254000
1,1,16,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254000
2,1,25,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254000
3,1,35,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254000
4,1,45,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254000
5,1,55,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254000
6,2,1,2.0,9.0,5.0,7.5,10.5,15.0,10.5,16.5,23.5,0.998333,0.998333,0.998333,0.998333,0.3750,-0.1250,0.3125,0.8750,1.059998,-1.410004,-0.350006,1.059998,1.016000
7,2,6,2.0,26.5,22.5,25.5,31.5,26.5,26.5,28.5,32.0,1.001667,0.981667,0.998333,1.005000,0.0625,-0.1875,0.2500,0.6875,NaN,NaN,NaN,1.409988,1.016000
8,2,11,2.0,21.5,15.5,20.5,25.0,26.5,23.5,25.0,27.0,1.001667,0.995000,0.998333,1.001667,0.3125,-0.0625,0.3125,0.6250,0.349991,NaN,-0.350006,1.759994,1.016000
9,2,16,2.0,18.0,14.0,17.5,21.0,20.5,18.0,20.5,23.0,0.995000,0.995000,0.998333,1.001667,0.2500,0.1250,0.3750,0.6875,0.349991,-1.059998,0.000000,1.059998,1.016000


DATA PREPROCESSING

In [6]:
#CONVERT TO FLOAT
for column in train_data.columns:
    train_data[column] = train_data[column].astype(np.float32)
train_data['Id'] = train_data['Id'].astype(np.int32)
train_data.head()

,Id,minutes_past,radardist_km,Ref,Ref_5x5_10th,Ref_5x5_50th,Ref_5x5_90th,RefComposite,RefComposite_5x5_10th,RefComposite_5x5_50th,RefComposite_5x5_90th,RhoHV,RhoHV_5x5_10th,RhoHV_5x5_50th,RhoHV_5x5_90th,Zdr,Zdr_5x5_10th,Zdr_5x5_50th,Zdr_5x5_90th,Kdp,Kdp_5x5_10th,Kdp_5x5_50th,Kdp_5x5_90th,Expected
0,1,3.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
1,1,16.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
2,1,25.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
3,1,35.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
4,1,45.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254


In [7]:
#REMOVING THE ROWS WITH NAN IN REF COLUMN
train_raw_tmp = train_data[~np.isnan(train_data.Ref)]
raw_ids_tmp = train_raw_tmp["Id"].unique()
train_new = train_data[np.in1d(raw_ids_all, raw_ids_tmp)]
train_new.shape

(2002634, 24)

In [6]:
#FILLING THE REST OF THE VALUES WITH ZERO
train_new = train_new.fillna(0.0)
train_new.head(10)

,Id,minutes_past,radardist_km,Ref,Ref_5x5_10th,Ref_5x5_50th,Ref_5x5_90th,RefComposite,RefComposite_5x5_10th,RefComposite_5x5_50th,RefComposite_5x5_90th,RhoHV,RhoHV_5x5_10th,RhoHV_5x5_50th,RhoHV_5x5_90th,Zdr,Zdr_5x5_10th,Zdr_5x5_50th,Zdr_5x5_90th,Kdp,Kdp_5x5_10th,Kdp_5x5_50th,Kdp_5x5_90th,Expected
6,2,1.0,2.0,9.0,5.0,7.5,10.5,15.0,10.5,16.5,23.5,0.998333,0.998333,0.998333,0.998333,0.3750,-0.1250,0.3125,0.8750,1.059998,-1.410004,-0.350006,1.059998,1.016001
7,2,6.0,2.0,26.5,22.5,25.5,31.5,26.5,26.5,28.5,32.0,1.001667,0.981667,0.998333,1.005000,0.0625,-0.1875,0.2500,0.6875,0.000000,0.000000,0.000000,1.409988,1.016001
8,2,11.0,2.0,21.5,15.5,20.5,25.0,26.5,23.5,25.0,27.0,1.001667,0.995000,0.998333,1.001667,0.3125,-0.0625,0.3125,0.6250,0.349991,0.000000,-0.350006,1.759995,1.016001
9,2,16.0,2.0,18.0,14.0,17.5,21.0,20.5,18.0,20.5,23.0,0.995000,0.995000,0.998333,1.001667,0.2500,0.1250,0.3750,0.6875,0.349991,-1.059998,0.000000,1.059998,1.016001
10,2,21.0,2.0,24.5,16.5,21.0,24.5,24.5,21.0,24.0,28.0,0.998333,0.995000,0.998333,0.998333,0.2500,0.0625,0.1875,0.5625,-0.350006,-1.059998,-0.350006,1.759995,1.016001
11,2,26.0,2.0,12.0,12.0,16.0,20.0,16.5,17.0,19.0,21.0,0.998333,0.995000,0.998333,0.998333,0.5625,0.2500,0.4375,0.6875,-1.760010,-1.760010,-0.350006,0.709991,1.016001
12,2,31.0,2.0,22.5,19.0,22.0,25.0,26.0,23.5,25.5,27.5,0.998333,0.995000,0.998333,1.001667,0.0000,-0.1875,0.2500,0.6250,-1.059998,-2.120010,-0.710007,0.349991,1.016001
13,2,37.0,2.0,14.0,14.0,18.5,21.0,19.5,20.0,21.0,23.0,0.998333,0.991667,0.998333,0.998333,0.5000,0.1875,0.4375,0.8125,0.000000,-1.760010,-0.350006,1.059998,1.016001
14,2,42.0,2.0,12.0,11.0,12.5,17.0,19.5,18.0,21.0,23.0,0.998333,0.995000,0.998333,0.998333,0.6250,0.3750,0.6250,0.8750,-0.350006,-0.350006,0.000000,0.349991,1.016001
15,2,47.0,2.0,1.5,3.5,7.0,10.5,18.0,16.5,18.5,21.5,0.998333,0.995000,0.998333,0.998333,0.3750,0.1875,0.5000,0.6875,0.349991,-2.110001,-0.350006,1.059998,1.016001


In [7]:
#ALL THE MISSING VALUES ARE HANDLED
train_new.isnull().sum()

Id                       0
minutes_past             0
radardist_km             0
Ref                      0
Ref_5x5_10th             0
Ref_5x5_50th             0
Ref_5x5_90th             0
RefComposite             0
RefComposite_5x5_10th    0
RefComposite_5x5_50th    0
RefComposite_5x5_90th    0
RhoHV                    0
RhoHV_5x5_10th           0
RhoHV_5x5_50th           0
RhoHV_5x5_90th           0
Zdr                      0
Zdr_5x5_10th             0
Zdr_5x5_50th             0
Zdr_5x5_90th             0
Kdp                      0
Kdp_5x5_10th             0
Kdp_5x5_50th             0
Kdp_5x5_90th             0
Expected                 0
dtype: int64

In [8]:
#REMOVE THE ROWS WITH EXPECTED GREATER THAN THRESHOLD=73mm
train_new_group = train_new.groupby('Id')
df = pd.DataFrame(train_new_group['Expected'].mean()) # mean, or any value
meaningful_ids = np.array(df[df['Expected'] < 73].index)
train_final = train_new[np.in1d(train_new.Id, meaningful_ids)]
train_final.shape

(1961311, 24)

In [9]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
train_final.iloc[:,:23]=sc.fit_transform(train_final.iloc[:,:23])
train_final.head()  #z=x-u/s

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


,Id,minutes_past,radardist_km,Ref,Ref_5x5_10th,Ref_5x5_50th,Ref_5x5_90th,RefComposite,RefComposite_5x5_10th,RefComposite_5x5_50th,RefComposite_5x5_90th,RhoHV,RhoHV_5x5_10th,RhoHV_5x5_50th,RhoHV_5x5_90th,Zdr,Zdr_5x5_10th,Zdr_5x5_50th,Zdr_5x5_90th,Kdp,Kdp_5x5_10th,Kdp_5x5_50th,Kdp_5x5_90th,Expected
6,-1.77416,-1.647435,-1.930281,-0.528447,-0.559557,-0.636787,-0.774702,-0.245660,-0.286110,-0.127215,-0.006049,0.982915,1.249227,0.979386,0.744440,0.082969,0.250784,0.180564,-0.243342,0.412988,-0.052050,-0.104647,-0.312249,1.016001
7,-1.77416,-1.358818,-1.930281,0.752664,0.884440,0.708412,0.698492,0.562168,0.927615,0.730511,0.579015,0.989717,1.213457,0.979386,0.757946,-0.197600,0.167589,0.091206,-0.358325,-0.007465,0.545678,0.131872,-0.213773,1.016001
8,-1.77416,-1.070201,-1.930281,0.386632,0.306841,0.334746,0.242504,0.562168,0.700042,0.480341,0.234860,0.989717,1.242073,0.979386,0.751193,0.026855,0.333979,0.180564,-0.396653,0.131361,0.545678,-0.104647,-0.115292,1.016001
9,-1.77416,-0.781584,-1.930281,0.130410,0.183070,0.110546,-0.038105,0.140692,0.282824,0.158694,-0.040464,0.976112,1.242073,0.979386,0.751193,-0.029259,0.583563,0.269922,-0.358325,0.131361,0.096324,0.131872,-0.312249,1.016001
10,-1.77416,-0.492967,-1.930281,0.606251,0.389355,0.372112,0.207428,0.421676,0.510397,0.408864,0.303691,0.982915,1.242073,0.979386,0.744440,-0.029259,0.500368,0.001848,-0.434981,-0.146296,0.096324,-0.104647,-0.115292,1.016001


In [10]:
#PADDING ZEROS TO CONVERT VARIABLE SIZED VECTOR INTO FIXED SIZE VECTOR
def pad_series(X, target_len=19):
    seq_len = X.shape[0]
    pad_size = target_len-seq_len
    if (pad_size > 0):
        X = np.pad(X, ((0,pad_size), (0,0)), 'constant', constant_values=0.)
    return X, seq_len

In [11]:
data_pd_gp = train_final.groupby("Id")
data_size = len(data_pd_gp)

In [12]:
#CONVERTING 2D DATASET TO 3D TO FEED LSTM MODEL
INPUT_WIDTH = 19

X_train = np.empty((data_size, INPUT_WIDTH, 22))
seq_lengths = np.zeros(data_size)
y_train = np.zeros(data_size)


i = 0
for _, group in data_pd_gp:
    group_array = np.array(group)
    X, seq_length = pad_series(group_array[:,1:23], target_len=INPUT_WIDTH) 
    y = group_array[0,23]
    X_train[i,:,:] = X[:,:]
    seq_lengths[i] = seq_length
    y_train[i]= y
    i += 1
    
X_train.shape, y_train.shape

((157580, 19, 22), (157580,))

ARCHITECTURE OF LSTM MODEL

In [13]:
from keras.layers import (
    Input,
    Dense,
    LSTM,
    GlobalAveragePooling1D,
    AveragePooling1D,
    TimeDistributed,
    Flatten,
    Bidirectional,
    Dropout,
    Masking,
    Layer,
    BatchNormalization
)
from keras.models import Model
from keras.optimizers import Adam,Nadam

In [14]:
#CREATING NON MASKING BEFORE FLATTEN
class NonMasking(Layer):   
    def __init__(self, **kwargs):   
        self.supports_masking = True  
        super(NonMasking, self).__init__(**kwargs)   
    def build(self, input_shape):   
        input_shape = input_shape   
    def compute_mask(self, input, input_mask=None):   
        # do not pass the mask to the next layers   
        return None   
    def call(self, x, mask=None):   
        return x   
    def get_output_shape_for(self, input_shape):   
        return input_shape 

In [16]:
def get_model_deep(shape=(19,22)):
    inp = Input(shape)
    x = Dense(16)(inp)
    x = BatchNormalization()(x)
    x = Masking(mask_value=0.)(x)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = BatchNormalization()(x)
    x = TimeDistributed(Dense(64))(x)
    x = BatchNormalization()(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = BatchNormalization()(x)
    x = TimeDistributed(Dense(1))(x)
    x = BatchNormalization()(x)
    x = NonMasking()(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(1)(x)

    model = Model(inp, x)
    return model

COMPILE THE MODEL

In [17]:
model = get_model_deep((19,22))
model.compile(optimizer=Nadam(learning_rate=0.001), loss='mae')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


TRAIN THE MODEL

In [18]:
model.fit(X_train, y_train, batch_size=64, epochs=1)

2463/2463 [==============================] - 357s 128ms/step - loss: 2.5783


In [19]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 19, 22)]          0         
_________________________________________________________________
dense (Dense)                (None, 19, 16)            368       
_________________________________________________________________
batch_normalization (BatchNo (None, 19, 16)            64        
_________________________________________________________________
masking (Masking)            (None, 19, 16)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 19, 128)           41472     
_________________________________________________________________
batch_normalization_1 (Batch (None, 19, 128)           512       
_________________________________________________________________
time_distributed (TimeDistri (None, 19, 64)            8256  

SAVE THE MODEL AND WEIGHTS

In [21]:
from keras.models import load_model
model.save('masked_lastm.h5')
model.save_weights("ml_weights.h5")

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [23]:
del train_data
del raw_ids_all
del raw_ids
del train_raw_tmp
del raw_ids_tmp
del train_new
del train_new_group
del df
del meaningful_ids
del train_final
del data_pd_gp
del data_size
del X_train
del y_train
del seq_lengths

LOAD TEST SET

In [25]:
test_raw = pd.read_csv('/content/drive/MyDrive/test.csv',nrows=2e6)
test_raw_ids_all = test_raw["Id"]
test_raw_ids = np.array(test_raw_ids_all.unique())

PREPROCESS TEST DATA

In [26]:
#FILL MISSING VALUES WITH ZEROS
test_new = test_raw.fillna(0.0)
#SCALE THE TEST DATA
test_new.iloc[:,:23]=sc.transform(test_new.iloc[:,:23])

In [27]:
#CONVERT 2D TO 3D
INPUT_WIDTH = 19

data_pd_gp = test_new.groupby("Id")
data_size = len(data_pd_gp)
print(test_new.shape)

X_test = np.empty((data_size, INPUT_WIDTH, 22))
seq_lengths = np.zeros(data_size)

i = 0
for _, group in data_pd_gp:
    group_array = np.array(group)
    X, seq_length = pad_series(group_array[:,1:23], target_len=INPUT_WIDTH) 
    X_test[i,:,:] = X[:,:]
    seq_lengths[i] = seq_length
    i += 1
    
X_test.shape

(2000000, 23)


(180880, 19, 22)

PREDICT TEST DATA

In [28]:
output = model.predict(X_test, batch_size=64,verbose=1)

2827/2827 [==============================] - 113s 38ms/step


In [44]:
output[:4]
print(len(output))
for i in range(len(output)):
  if output[i] <0:
    output[i]=0

180880


In [45]:
my_submission = pd.DataFrame({'Id': np.arange(1,output.shape[0]+1), 'Expected': output[:,0]})

In [46]:
my_submission.tail()

,Id,Expected
180875,180876,1.279909
180876,180877,8.061604
180877,180878,1.350357
180878,180879,1.279840
180879,180880,2.285069


In [47]:
actual=pd.read_csv('/content/drive/MyDrive/sample_solution.csv',nrows=180880)
print(len(actual))
actual.tail()

180880


,Id,Expected
180875,180876,3.344117
180876,180877,27.577250
180877,180878,1.468554
180878,180879,3.349743
180879,180880,5.312242


In [48]:
act=actual["Expected"]
predicted=my_submission["Expected"]

ERROR CALCULATION

In [50]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(act,predicted)

1.258304593338303

CONCATENATION OF RESULTS FROM LSTM AND RIDGE REGRESSION

In [51]:
reg_output=pd.read_csv('/content/drive/MyDrive/sub (1).csv',nrows=180880)
reg_out=reg_output['Expected'].values
rnn_out=my_submission['Expected'].values

In [52]:
for i in range(len(reg_out)):
  if rnn_out[i]<5:
    rnn_out[i]=(rnn_out[i]+reg_out[i])/2

In [53]:
my_submission["Expected"]=rnn_out
my_submission.tail()

,Id,Expected
180875,180876,3.208961
180876,180877,8.061604
180877,180878,2.433403
180878,180879,3.101017
180879,180880,3.211053


In [54]:
act_combined=actual["Expected"]
predicted_combined=my_submission["Expected"]

ERROR OF LSTM-REG 

In [55]:
mean_absolute_error(act_combined,predicted_combined)

1.082132572887785